### SQLite

오픈 소스, 가벼운 유형으로 모든 시스템에서 사용이 가능하다.

데이터 형식의 다양성(스키마 지정), 수평 확장성(데이터베이스 네트워크 간 확장), 데이터 관리의 엄격성에서는 다소 약점이 있고 이런 부분에서는 MySQL이 강점을 가지고 있다.

> 실제 상황과의 괴리 : 실제는 서버에서 돌아가고, 대용량 데이터가 여러 데이터베이스간 교차하여 작동함.

지원하는 데이터 유형

* TEXT 문자열(UTF-8, UTF-16BE, UTF16LE)
* REAL 8바이트 부동 소수점 값. float32
* NULL
* INTEGER 8바이트 정수값. int32
* BLOB 입력형태 그대로 저장(이미지, 오디오, 멀티미디어 파일)

In [1]:
install.packages("DBI")
install.packages("RSQLite")
library(DBI)
library(RSQLite)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [8]:
conn = dbConnect(RSQLite::SQLite(), "CarsDB.db")  ## create DB file of current working space

> 작업공간에 데이터베이스 파일 생성, conn으로 호출

In [6]:
data("mtcars")
mtcars$car_names = rownames(mtcars)
rownames(mtcars) = NULL
head(mtcars)

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,car_names
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4,Mazda RX4
2,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4,Mazda RX4 Wag
3,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1,Datsun 710
4,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1,Hornet 4 Drive
5,18.7,8,360,175,3.15,3.440,17.02,0,0,3,2,Hornet Sportabout
6,18.1,6,225,105,2.76,3.460,20.22,1,0,3,1,Valiant


In [9]:
dbWriteTable(conn, "cars_data", mtcars)  ## write relation, table name : cars_data
dbListTables(conn)  ## call DB relation list

[1] "cars_data"

In [10]:
dbDisconnect(conn)  ## disconnecting database

`-` 기초 예제

In [11]:
conn = dbConnect(SQLite(), "CarsDB.db")  ## Connecting DB file

In [12]:
## create dataframe
car <- c('Camaro', 'California', 'Mustang', 'Explorer')
make <- c('Chevrolet','Ferrari','Ford','Ford')
df1 <- data.frame(car,make)

car <- c('Corolla', 'Lancer', 'Sportage', 'XE')
make <- c('Toyota','Mitsubishi','Kia','Jaguar')
df2 = data.frame(car, make)
dfList = list(df1, df2)

for (k in 1:length(dfList)) {
    dbWriteTable(conn, "Cars_and_Makes",
                 dfList[[k]], append = TRUE)
}  ## Write two dataframes into the Cars_and_Makes relation.

In [13]:
dbListTables(conn)

dbGetQuery(conn, "SELECT * FROM Cars_and_Makes")  ## SELECT all(*) FROM Cars_and_Makes relation
dbGetQuery(conn, "SELECT * FROM cars_data LIMIT 20")  ## SELECT all(*) columns FROM carsr_data relation.
# maximun 20 tuples
dbGetQuery(conn, "SELECT car_names, hp, cyl FROM cars_data WHERE cyl = 8")

[1] "Cars_and_Makes" "cars_data"

car,make
<chr>,<chr>
Camaro,Chevrolet
California,Ferrari
Mustang,Ford
Explorer,Ford
Corolla,Toyota
Lancer,Mitsubishi
Sportage,Kia
XE,Jaguar


mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,car_names
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4,Mazda RX4
21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4,Mazda RX4 Wag
22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1,Datsun 710
21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1,Hornet 4 Drive
18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2,Hornet Sportabout
18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1,Valiant
14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4,Duster 360
24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2,Merc 240D
22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2,Merc 230


car_names,hp,cyl
<chr>,<dbl>,<dbl>
Hornet Sportabout,175,8
Duster 360,245,8
Merc 450SE,180,8
Merc 450SL,180,8
Merc 450SLC,180,8
Cadillac Fleetwood,205,8
Lincoln Continental,215,8
Chrysler Imperial,230,8
Dodge Challenger,150,8


In [26]:
dbGetQuery(conn, "SELECT cyl, AVG(hp) FROM cars_data")

cyl,AVG(hp)
<dbl>,<dbl>
6,146.6875


In [20]:
dbGetQuery(conn, "SELECT cyl, AVG(hp) AS 'average_hp', AVG(mpg) AS 'average_mpg' FROM cars_data GROUP BY cyl ORDER BY average_mpg")

cyl,average_hp,average_mpg
<dbl>,<dbl>,<dbl>
8,209.21429,15.10000
6,122.28571,19.74286
4,82.63636,26.66364


SELECT cyl, AVG(hp), AVG(mpg) : hp와 mpg는 평균을 내어 산출

AS 'average_hp', AS 'average_mpg' : 해당 열 이름으로

FROM cars_data

GROUP BY cyl : cyl의 값들을 기준으로 그룹화하여 계산

SELECT car_names, mpg, hp, wt, FROM cars_data WHERE mpg >= ? AND hp >= ? AND wt >= ? : WHERE 구문에 AND, OR 사용 가능